# Interacción Agente-a-Agente (A2A): Consultor Técnico Aeroespacial Remoto

## Descripción

En este ejercicio, experimentarás con **comunicación Agente-a-Agente (A2A)** - un patrón donde un agente local se comunica con un agente remoto alojado en un servidor. Este demo integra:

### Componentes del Sistema

1. **Agente Remoto** (en `remote-agent/agent.py`)
   - Nombre: "technical-support-agent"
   - Descripción: "A technical support agent for plane engineering"
   - Especialización: Ingeniería aeroespacial y soporte técnico
   - Alojado en servidor FastAPI en `http://127.0.0.1:8000`
   - Expone endpoint: `/.well-known/agent.json` (metadatos)
   - Expone endpoint: `/invoke` (procesamiento de consultas)

2. **Agente Local** (este notebook)
   - Se comunica con el agente remoto vía HTTP
   - Recupera tarjeta de agente (metadata)
   - Envía preguntas técnicas
   - Procesa respuestas

### Flujo de Ejecución
```
Pregunta técnica
    ↓
[Agente Local]
    ↓
[A2ACardResolver]
    ↓
http://127.0.0.1:8000/.well-known/agent.json
    ↓
[A2AAgent]
    ↓
http://127.0.0.1:8000/invoke
    ↓
[Agente Remoto - Azure AI Foundry]
    ↓
Respuesta técnica
```

### Requisitos Previos
- El servidor remoto debe estar ejecutándose: `python remote-agent/agent.py`
- Configuración de Azure AI Foundry completada (PROJECT_ENDPOINT, MODEL_DEPLOYMENT_NAME)
- Protocolo HTTP disponible en localhost:8000

### Concepto de A2A
El protocolo A2A permite que agentes independientes colaboren entre sí, cada uno con su propia:
- Lógica de negocio
- Almacenamiento de datos
- Configuración de LLM
- Localización (local o remota)

## Inicio del Demo

Este notebook demuestra una **interacción Agente-a-Agente (A2A)** entre:
- Un **agente local** (este notebook) que actúa como cliente
- Un **agente remoto** (servidor FastAPI) que actúa como especialista técnico aeroespacial

### Configuración del Agente Remoto
El servidor remoto (`remote-agent/agent.py`) expone un agente de Azure AI Foundry con:
- **Nombre**: technical-support-agent
- **Función**: Responder preguntas técnicas sobre ingeniería aeroespacial
- **Datos de Configuración** (desde `agent.json`):
  ```json
  {
    "name": "technical-support-agent",
    "description": "A technical support agent for plane engineering",
    "url": "http://127.0.0.1:8000/invoke",
    "version": "1.0",
    "capabilities": {"text": "text"}
  }
  ```

### Flujo de Comunicación
1. El agente local recupera los metadatos del agente remoto
2. Crea una instancia de A2AAgent con la tarjeta del agente
3. Envía preguntas técnicas al endpoint `/invoke`
4. Recibe y procesa respuestas del agente remoto

## 1. Importar Librerías

### Dependencias del Demo A2A
- **httpx**: Cliente HTTP asincrónico para comunicación con el servidor remoto
- **A2ACardResolver**: Recupera los metadatos (tarjeta) del agente remoto desde el endpoint `.well-known/agent.json`
- **A2AAgent**: Crea una instancia de agente que se comunica con el servidor remoto

In [ ]:
import httpx
from a2a.client import A2ACardResolver
from agent_framework.a2a import A2AAgent

## 2. Inicializar Comunicación A2A

### Función `main()`
1. Crea un cliente HTTP asincrónico con timeout de 60 segundos
2. Inicializa `A2ACardResolver` para recuperar metadatos del agente remoto
3. Obtiene la tarjeta de agente desde `http://127.0.0.1:8000/.well-known/agent.json`
4. Crea una instancia `A2AAgent` con los metadatos recuperados
5. Crea un thread de conversación y envía preguntas técnicas
6. Procesa y muestra respuestas

### Manejo de Errores
- `httpx.ConnectError`: Verifica que el servidor remoto esté ejecutándose en puerto 8000
- `Exception`: Errores generales durante la comunicación A2A

In [ ]:
async def main():
    # Create an async HTTP client for communication
    async with httpx.AsyncClient(timeout=60.0) as http_client:
        # Initialize the A2A card resolver to retrieve remote agent metadata
        resolver = A2ACardResolver(
            httpx_client=http_client, base_url="http://127.0.0.1:8000"
        )
        try:
            print("Retrieving agent card from remote agent...")
            agent_card = await resolver.get_agent_card(
                relative_card_path="/.well-known/agent.json"
            )
            print(f"   Agent Name: {agent_card.name}")
            print(f"   Description: {agent_card.description}")
            print(f"   URL: http://127.0.0.1:8000/invoke")
            print("-"*80 + "\n")
            
            # Create an A2A agent instance with the retrieved card
            agent = A2AAgent(
                name=agent_card.name,   
                description=agent_card.description,
                agent_card=agent_card,
                url="http://127.0.0.1:8000/invoke",
            )

            # Create a new conversation thread
            thread = agent.get_new_thread()
            
            # Define the question to send to the remote agent
            question = "¿Qué medidas deben tomarse respecto al torque de nivel según la JSSG?"
            print(f"Sending question to remote agent...")
            
            print("Waiting for remote agent response...\n")
            # Send the question and wait for the response
            response = await agent.run(question, thread=thread, http_client=http_client)

            print("RESPONSE FROM REMOTE AGENT:")
            print("="*80)
            for message in response.messages:
                print(message.text)
            print("="*80 + "\n")

        except httpx.ConnectError as e:
            print(f"Connection error: {e}")
            print("   Make sure the remote agent server is running on http://127.0.0.1:8000")
        except Exception as e:
            print(f"An error occurred: {e}")

## 3. Ejecutar el Demo A2A

### Preguntas Técnicas Ejemplares
El agente remoto responde preguntas sobre especificaciones aeroespaciales, normas técnicas y procedimientos. Algunas ejemplos:

**Pregunta actual**: "¿Qué medidas deben tomarse respecto al torque de nivel según la JSSG?"
- JSSG: Especificación General de Sistemas Conjuntos (Joint Service Specification)
- Consulta técnica sobre procedimientos de torsión en componentes aeronáuticos

### Flujo Esperado
1. El agente local se conecta al servidor en `http://127.0.0.1:8000`
2. Recupera metadatos del agente remoto
3. Envía la pregunta técnica
4. Agente remoto procesa usando Azure AI Foundry
5. Respuesta se muestra en el notebook

### Requisitos de Ejecución
- ✅ Servidor remoto ejecutándose: `python remote-agent/agent.py`
- ✅ Variables de entorno configuradas (`.env`)
- ✅ Autenticación Azure CLI funcional

In [ ]:
await main()